In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error, accuracy_score, precision_score,recall_score, f1_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
#filtered and merged data
jd = pd.read_csv('jd.csv')
jd.head()

,Unnamed: 0,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [3]:
jd = jd.drop(columns=['Transaction unique identifier', 'ADDRESS','Unnamed: 0'])
jd.shape

(115372, 51)

In [4]:
#one hot encdoing on all categorical data
one_hot_encoded_data = pd.get_dummies(jd, columns = ['Postcode', 'Property Type', 'Old/New' , 'Duration', 'Street_x', 'PAON', 'District', 'County',
          'PPD Category Type', 'CURRENT_ENERGY_RATING', 'POTENTIAL_ENERGY_RATING', 'PROPERTY_TYPE', 'BUILT_FORM',
          'CONSTITUENCY', 'TRANSACTION_TYPE', 'ENERGY_TARIFF', 'HOT_WATER_ENERGY_EFF', 'HOT_WATER_ENV_EFF', 'WINDOWS_ENERGY_EFF',
           'WINDOWS_ENV_EFF','WALLS_ENERGY_EFF','WALLS_ENV_EFF','MAINHEAT_ENERGY_EFF','MAINHEAT_ENV_EFF','MAINHEATC_ENERGY_EFF',
           'MAINHEATC_ENV_EFF','LIGHTING_ENERGY_EFF','LIGHTING_ENV_EFF','MAIN_FUEL','CONSTITUENCY_LABEL'])
print(one_hot_encoded_data)


         Price  CURRENT_ENERGY_EFFICIENCY  POTENTIAL_ENERGY_EFFICIENCY  \
0        35000                         58                           75   
1        33000                         58                           75   
2        54650                         58                           75   
3       115000                         58                           75   
4        36000                         73                           76   
...        ...                        ...                          ...   
115367  119500                         64                           77   
115368  119500                         70                           71   
115369  190000                         57                           84   
115370  110000                         56                           78   
115371  110000                         58                           70   

        ENVIRONMENT_IMPACT_CURRENT  ENVIRONMENT_IMPACT_POTENTIAL  \
0                               52         

In [5]:
one_hot_encoded_data.shape

(115372, 8333)

In [6]:
one_hot_encoded_data =one_hot_encoded_data.loc[:,~one_hot_encoded_data.columns.duplicated()]
one_hot_encoded_data.shape

(115372, 8333)

In [7]:
train_df = one_hot_encoded_data[one_hot_encoded_data['Year'] > 2016]
train_df.shape

(20756, 8333)

In [8]:
test_df = one_hot_encoded_data[one_hot_encoded_data['Year'] <= 2016]
test_df.shape

(94616, 8333)

In [9]:
X_train = train_df.drop(['Price'],axis=1)
y_train = train_df['Price']
X_test= test_df.drop(['Price'],axis=1)
y_test= test_df['Price']

In [10]:
modelXGB1 = XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1.0, colsample_bytree=1.0)
modelXGB1.fit(X_train, y_train)
r_sqXGB1 = modelXGB1.score(X_train, y_train)
print('coefficient of determination for XGB:', r_sqXGB1)
y_predXGB1 = modelXGB1.predict(X_test)
MAE_XGB1 = mean_absolute_error(y_test, y_predXGB1)
print('MAE for XGB is:', MAE_XGB1)

MSE_XGB1 = mean_squared_error(y_test, y_predXGB1)
print('MSE for XGB is:', MSE_XGB1)
RMSE_XGB1 = sqrt(MSE_XGB1)
print('RMSEE for XGB is:', RMSE_XGB1)

coefficient of determination for XGB: 0.9145657867214635
MAE for XGB is: 73988.31616687783
MSE for XGB is: 8873313535.295454
RMSEE for XGB is: 94198.26715654304


In [11]:
model1 = LinearRegression()
model1.fit(X_train, y_train)
r_sq1= model1.score(X_train, y_train)
print('coefficient of determination:', r_sq1)
y_predLR1 = model1.predict(X_test)
MAE_LR1 = mean_absolute_error(y_test, y_predLR1)
print('MAE for LR is:', MAE_LR1)

MSE_LR1 = mean_squared_error(y_test, y_predLR1)
print('MSE for LR is:', MSE_LR1)
RMSE_LR1 = sqrt(MSE_LR1)
print('RMSEE for LR is:', RMSE_LR1)
    

coefficient of determination: 0.7171885837259122
MAE for LR is: 23271418777.742054
MSE for LR is: 2.82690089069193e+22
RMSEE for LR is: 168133901718.00363


In [12]:
modelRM1 = RandomForestClassifier(n_estimators=100)
modelRM1.fit(X_train, y_train)
r_sqRM1 = modelRM1.score(X_train, y_train)
print('coefficient of determination:', r_sqRM1)
y_predRM1 = modelRM1.predict(X_test)
MAE_RM1 = mean_absolute_error(y_test, y_predRM1)
print('MAE for RM is:', MAE_RM1)

MSE_RM1 = mean_squared_error(y_test, y_predRM1)
print('MSE for RM is:', MSE_RM1)
RMSE_RM1 = sqrt(MSE_RM1)
print('RMSEE for RM is:', RMSE_RM1)
    

coefficient of determination: 0.9968201965696666
MAE for RM is: 78400.69389955187
MSE for RM is: 12657972079.821108
RMSEE for RM is: 112507.65342776067


In [13]:
modelLASSO1 = Lasso(alpha=0.01)
modelLASSO1.fit(X_train, y_train)
r_sqLASSO1 = modelLASSO1.score(X_train, y_train)
print('coefficient of determination for LASSO:', r_sqLASSO1)
y_predLASSO1 = modelLASSO1.predict(X_test)
MAE_LASSO1 = mean_absolute_error(y_test, y_predLASSO1)
print('MAE for LASSO is:', MAE_LASSO1)

MSE_LASSO1 = mean_squared_error(y_test, y_predLASSO1)
print('MSE for LASSO is:', MSE_LASSO1)
RMSE_LASSO1 = sqrt(MSE_LASSO1)
print('RMSEE for LASSO is:', RMSE_LASSO1)

C:\Users\gpathak2\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 56711532086451.06, tolerance: 40168914537.520805
  model = cd_fast.enet_coordinate_descent(


coefficient of determination for LASSO: 0.7171878482256459
MAE for LASSO is: 43754.30968745027
MSE for LASSO is: 5719765440.001168
RMSEE for LASSO is: 75629.13089545039


In [14]:
####Creating the list of output. Lasso model was running for hours without giving any particular result or even an error.
##Without any error it was difficult to identify the issue in the running code. So Excluded Lasso from the result.

model = ['Linear Regression','Random Forest', 'XGBOOST', 'LASSO']
R_SQUARED = [r_sq1, r_sqRM1, r_sqXGB1, r_sqLASSO1]
MAE = [MAE_LR1,MAE_RM1,MAE_XGB1,MAE_LASSO1]
MSE = [MSE_LR1,MSE_RM1,MSE_XGB1,MSE_LASSO1]
RMSE = [RMSE_LR1,RMSE_RM1,RMSE_XGB1, RMSE_LASSO1]

In [15]:
Result1 = pd.DataFrame()
Result1['Model'] = model
Result1['R_SQUARED'] = R_SQUARED
Result1['MAE'] = MAE
Result1['MSE'] = MSE
Result1['RMSE'] = RMSE
Result1

,Model,R_SQUARED,MAE,MSE,RMSE
0,Linear Regression,0.717189,2.327142e+10,2.826901e+22,1.681339e+11
1,Random Forest,0.996820,7.840069e+04,1.265797e+10,1.125077e+05
2,XGBOOST,0.914566,7.398832e+04,8.873314e+09,9.419827e+04
3,LASSO,0.717188,4.375431e+04,5.719765e+09,7.562913e+04


In [16]:
test_df = pd.read_csv('test_df.csv')
test_df.head()

,Unnamed: 0,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [17]:
test_df = test_df.drop(columns = ['Unnamed: 0'])
test_df.head()

,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,County,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,WEST MIDLANDS,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [18]:
test_df.shape

(94616, 53)

In [19]:
df_from_arr = pd.DataFrame(data=[y_test,y_predLR1, y_predRM1,y_predXGB1,y_predLASSO1])
pred_df1 = df_from_arr.T
pred_df1.shape

(94616, 5)

In [20]:
#dataframe of predicted y value
pred_df1.columns = ['y_test','y_predLR1', 'y_predRM1','y_predXGB1','y_predLASSO1']
pred_df1.head()

,y_test,y_predLR1,y_predRM1,y_predXGB1,y_predLASSO1
0,35000.0,22931.5625,150000.0,178851.171875,23141.943185
1,33000.0,49475.1250,150000.0,178851.171875,49690.722841
2,54650.0,54783.8125,150000.0,178851.171875,55000.478772
3,115000.0,76018.6250,150000.0,178851.171875,76239.502497
4,36000.0,-32842.4375,160000.0,243289.859375,-32709.117422


In [21]:
pred_df1.insert(0, 'ID', range(0, 0 + len(pred_df1)))
pred_df1.head()

,ID,y_test,y_predLR1,y_predRM1,y_predXGB1,y_predLASSO1
0,0,35000.0,22931.5625,150000.0,178851.171875,23141.943185
1,1,33000.0,49475.1250,150000.0,178851.171875,49690.722841
2,2,54650.0,54783.8125,150000.0,178851.171875,55000.478772
3,3,115000.0,76018.6250,150000.0,178851.171875,76239.502497
4,4,36000.0,-32842.4375,160000.0,243289.859375,-32709.117422


In [22]:
test_df.insert(0, 'ID', range(0, 0 + len(test_df)))
test_df.head()

,ID,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [23]:
#merging predicted value with test dataframe
test_df_One_hot = pd.merge(test_df, pred_df1, left_on = ['ID','Price'], right_on = ['ID','y_test'])
test_df_One_hot.shape

(94616, 59)

In [24]:
test_df_One_hot.head(20)

,ID,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year,y_test,y_predLR1,y_predRM1,y_predXGB1,y_predLASSO1
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995,35000.0,22931.5625,150000.0,178851.171875,23141.943185
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000,33000.0,49475.1250,150000.0,178851.171875,49690.722841
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001,54650.0,54783.8125,150000.0,178851.171875,55000.478772
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005,115000.0,76018.6250,150000.0,178851.171875,76239.502497
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995,36000.0,-32842.4375,160000.0,243289.859375,-32709.117422
5,5,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Very Good,mains gas (not community),"35, Lower Ford Street",Coventry South,1995,36000.0,-58209.0625,225000.0,298781.250000,-58071.059600
6,6,{C097F7F4-6E62-41F0-BC30-46C797A4FA6D},12000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995,12000.0,-32842.4375,160000.0,243289.859375,-32709.117422
7,7,{C097F7F4-6E62-41F0-BC30-46C797A4FA6D},12000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Very Good,mains gas (not community),"35, Lower Ford Street",Coventry South,1995,12000.0,-58209.0625,225000.0,298781.250000,-58071.059600
8,8,{460DDE15-4025-411F-8F2F-F59BEC3FB597},65000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,2004,65000.0,14935.9375,160000.0,243289.859375,15078.685959
9,9,{460DDE15-4025-411F-8F2F-F59BEC3FB597},65000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Very Good,mains gas (not community),"35, Lower Ford Street",Coventry South,2004,65000.0,-10430.6875,225000.0,298781.250000,-10283.256220


In [25]:
test_df_One_hot.to_csv('test_df_One_hot2.csv')

In [55]:
######################################################################################
#Modelling Without year attributes
###################################################################

In [57]:
New1 = one_hot_encoded_data.copy() 
New1.shape

(115372, 8333)

In [58]:
train_df = New1[New1['Year'] > 2016]

test_df = New1[New1['Year'] <= 2016]


In [59]:
train_df = train_df.drop(columns=['Year'])
train_df.shape

(20756, 8332)

In [60]:
test_df = test_df.drop(columns=['Year'])
test_df.shape

(94616, 8332)

In [61]:
X_train = train_df.drop(['Price'],axis=1)
y_train = train_df['Price']
X_test= test_df.drop(['Price'],axis=1)
y_test= test_df['Price']

In [62]:
#linear regression model without year attributes
model = LinearRegression()
model.fit(X_train, y_train)
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)
y_predLR = model.predict(X_test)
MAE_LR = mean_absolute_error(y_test, y_predLR)
print('MAE for LR is:', MAE_LR)

MSE_LR = mean_squared_error(y_test, y_predLR)
print('MSE for LR is:', MSE_LR)
RMSE_LR = sqrt(MSE_LR)
print('RMSEE for LR is:', RMSE_LR)
    

coefficient of determination: 0.7156296672150905
MAE for LR is: 31622277172.589603
MSE for LR is: 4.451489518371626e+22
RMSEE for LR is: 210985533114.75235


In [63]:
df_from_arr = pd.DataFrame(data=[y_test, y_predLR])
pred_df = df_from_arr.T
pred_df.shape

(94616, 2)

In [64]:
pred_df.columns = ['y_test','y_predLR_New']
pred_df.head()

,y_test,y_predLR_New
0,35000.0,144196.796875
1,33000.0,144196.796875
2,54650.0,144196.796875
3,115000.0,144196.796875
4,36000.0,90211.640625


In [65]:
pred_df.insert(0, 'ID', range(0, 0 + len(pred_df)))
pred_df.head()

,ID,y_test,y_predLR_New
0,0,35000.0,144196.796875
1,1,33000.0,144196.796875
2,2,54650.0,144196.796875
3,3,115000.0,144196.796875
4,4,36000.0,90211.640625


In [66]:
test_df_One_hot1 = pd.merge(test_df_One_hot, pred_df, left_on = ['ID','y_test'], right_on = ['ID','y_test'])
test_df_One_hot1.shape


(94616, 59)

In [67]:
test_df_One_hot1.to_csv('test_df_One_hot1.csv')